In [2]:
# soc rtl source files
sv_files_path = "/home/cao/Projects/opentitan_cache/bazel/_bazel_dev/3d83422381937a86b10787a3f52f4c40/execroot/lowrisc_opentitan/bazel-out/k8-fastbuild/bin/hw/build.verilator_real/src"

import pathlib

def get_modules_with_rtl(root_path, max_depth=5):
    """
    搜索含有rtl目录的模块，并获取每个模块rtl目录下所有的.sv文件
    
    Args:
        root_path (str): 搜索的根路径
        max_depth (int): 最大搜索深度
        
    Returns:
        dict: 模块名(rtl目录的父目录名)为key，该模块rtl下所有.sv文件的绝对路径列表为value的字典
    """
    rtl_modules_sv_files = {}
    rtl_paths = []
    root = pathlib.Path(root_path)
    
    def find_rtl_dirs(path, current_depth=1):
        if current_depth > max_depth:
            return
        
        try:
            for item in path.iterdir():
                if item.is_dir():
                    if item.name == 'rtl':
                        # 找到rtl目录，将其父目录名作为模块名
                        module_name = path.name
                        module_sv_files = []
                        
                        # 搜集rtl目录下所有的.sv文件
                        for sv_file in item.glob('**/*.sv'):
                            path = str(sv_file.absolute())
                            module_sv_files.append(path)
                            rtl_paths.append(path)       
                        if module_sv_files:  # 只有当存在.sv文件时才添加到结果字典中
                            rtl_modules_sv_files[module_name] = module_sv_files
                
                    else:
                        find_rtl_dirs(item, current_depth + 1)
        except PermissionError:
            print(f"Permission denied: {path}")
        except Exception as e:
            print(f"Error accessing {path}: {e}")
    
    find_rtl_dirs(root)
    return rtl_modules_sv_files, rtl_paths

# 获取所有含有rtl目录的模块，以及其rtl目录下的所有.sv文件
rtl_modules_sv_files, rtl_paths = get_modules_with_rtl(sv_files_path, 5)

# 统计信息
total_modules = len(rtl_modules_sv_files)
total_sv_files = len(rtl_paths)

print(f"找到 {total_modules} 个含有rtl目录的模块")
print(f"总共找到 {total_sv_files} 个.sv文件")
print(rtl_paths)

    


找到 191 个含有rtl目录的模块
总共找到 643 个.sv文件
['/home/cao/Projects/opentitan_cache/bazel/_bazel_dev/3d83422381937a86b10787a3f52f4c40/execroot/lowrisc_opentitan/bazel-out/k8-fastbuild/bin/hw/build.verilator_real/src/lowrisc_constants_jtag_id_pkg_0/rtl/jtag_id_pkg.sv', '/home/cao/Projects/opentitan_cache/bazel/_bazel_dev/3d83422381937a86b10787a3f52f4c40/execroot/lowrisc_opentitan/bazel-out/k8-fastbuild/bin/hw/build.verilator_real/src/lowrisc_constants_top_pkg_0/rtl/top_pkg.sv', '/home/cao/Projects/opentitan_cache/bazel/_bazel_dev/3d83422381937a86b10787a3f52f4c40/execroot/lowrisc_opentitan/bazel-out/k8-fastbuild/bin/hw/build.verilator_real/src/lowrisc_ibex_ibex_pkg_0.1/rtl/ibex_pkg.sv', '/home/cao/Projects/opentitan_cache/bazel/_bazel_dev/3d83422381937a86b10787a3f52f4c40/execroot/lowrisc_opentitan/bazel-out/k8-fastbuild/bin/hw/build.verilator_real/src/lowrisc_ip_entropy_src_pkg_0.1/rtl/entropy_src_pkg.sv', '/home/cao/Projects/opentitan_cache/bazel/_bazel_dev/3d83422381937a86b10787a3f52f4c40/execroot

In [ ]:
from verible_syntax_tool import verible_verilog_syntax
import pickle, os, anytree
def parse_sv_files(sv_file_paths):
    """
    解析给定的.sv文件列表为语法树
    
    Args:
        sv_files (list): .sv文件路径列表
        
    Returns:
        dict: 文件名为key，语法树为value的字典
    """
    parser = verible_verilog_syntax.VeribleVerilogSyntax(executable = '/usr/bin/verible-verilog-syntax')
    return parser.parse_files(sv_file_paths)

if 'ast_trees' not in locals():
    if os.path.exists('parsertls.pkl'):
        ast_trees = pickle.load(open('../ast_trees.pkl', 'rb'))
    else:
        ast_trees = parse_sv_files(rtl_paths)
        pickle.dump(ast_trees, open('../ast_trees.pkl', 'wb'))
        

for f in rtl_paths:
    if 'pkg' not in f and 'top' in f.rpartition('/')[2]:
        mod = f
print(mod)
data = ast_trees[mod]

def filter_nodes(node: verible_verilog_syntax.Node):
    """
    过滤节点，返回符合指定要求的节点
    
    Args:
        node (anytree.Node): 当前节点
        
    Returns:
        bool: 如果节点类型匹配则返回True，否则返回False
    """
    return 'instant' in str(node).lower() and 'u_' in node.text and node.size > 30
    #return True
nodes = anytree.findall(data.tree, filter_nodes)
for mod in nodes[:5]:
    print(mod)
    print(mod.text)
    for pre, _, node in anytree.RenderTree(nodes[1]):
        print(f"\033[90m{pre}\033[0m{node.to_formatted_string()}")


/home/cao/Projects/opentitan_cache/bazel/_bazel_dev/3d83422381937a86b10787a3f52f4c40/execroot/lowrisc_opentitan/bazel-out/k8-fastbuild/bin/hw/build.verilator_real/src/lowrisc_systems_top_earlgrey_0.1/rtl/autogen/top_earlgrey.sv
[kInstantiationBase]
pinmux_jtag_breakout u_dft_tap_breakout (
    .req_i    (pinmux_aon_dft_jtag_req),
    .rsp_o    (pinmux_aon_dft_jtag_rsp),
    .tck_o    (),
    .trst_no  (),
    .tms_o    (),
    .tdi_o    (),
    .tdo_i    (1'b0),
    .tdo_oe_i (1'b0)
  )
[kInstantiationBase]
├── [kInstantiationType]
│   └── [kDataType]
│       ├── null
│       ├── [kLocalRoot]
│       │   └── [kUnqualifiedId]
│       │       ├── [SymbolIdentifier] @(43630-43634) 'uart'
│       │       └── [kActualParameterList]
│       │           ├── [#] @(43635-43636)
│       │           └── [kParenGroup]
│       │               ├── [(] @(43636-43637)
│       │               ├── [kActualParameterByNameList]
│       │               │   └── [kParamByName]
│       │               │      

In [3]:
# 分析语法树结构
print("语法树的根节点类型:", type(data.tree))
print("根节点标签:", data.tree.tag)
print("\n总共有以下顶级节点:")
for i, child in enumerate(data.tree.children):
    print(f"{i+1}. {child.tag}")

# 查看模块定义部分
module_nodes = anytree.findall(data.tree, lambda node: hasattr(node, 'tag') and node.tag == 'kModuleDeclaration')
if module_nodes:
    print("\n找到", len(module_nodes), "个模块定义节点")
    for i, module in enumerate(module_nodes):
        # 查找模块名称
        module_name = module.find(lambda n: hasattr(n, 'tag') and n.tag == 'SymbolIdentifier')
        print(f"\n模块 {i+1} 名称: {module_name.text if module_name else '未找到名称'}")
        
        # 查找模块端口定义
        port_list = module.find(lambda n: hasattr(n, 'tag') and n.tag == 'kPortDeclarationList')
        if port_list:
            print("端口列表:", port_list.tag)
            ports = port_list.find_all(lambda n: hasattr(n, 'tag') and n.tag == 'kPort')
            print(f"找到 {len(ports)} 个端口定义")
            if len(ports) > 0:
                # 显示前3个端口的信息
                for j, port in enumerate(ports[:3]):
                    port_name = port.find(lambda n: hasattr(n, 'tag') and n.tag == 'SymbolIdentifier')
                    print(f"  端口 {j+1}: {port_name.text if port_name else '未找到名称'}")
        
        # 查找模块内部的声明和语句
        module_items = module.find(lambda n: hasattr(n, 'tag') and n.tag == 'kModuleItemList')
        if module_items:
            declarations = module_items.find_all(lambda n: hasattr(n, 'tag') and 'Declaration' in n.tag)
            print(f"\n找到 {len(declarations)} 个声明")
            # 显示前3个声明的类型
            for j, decl in enumerate(declarations[:3]):
                print(f"  声明 {j+1} 类型: {decl.tag}")
            
            statements = module_items.find_all(lambda n: hasattr(n, 'tag') and ('Statement' in n.tag or 'Assignment' in n.tag))
            print(f"\n找到 {len(statements)} 个语句/赋值")
            # 显示前3个语句的类型
            for j, stmt in enumerate(statements[:3]):
                print(f"  语句 {j+1} 类型: {stmt.tag}")



语法树的根节点类型: <class 'verible_syntax_tool.verible_verilog_syntax.RootNode'>
根节点标签: kDescriptionList

总共有以下顶级节点:
1. kModuleDeclaration

找到 1 个模块定义节点

模块 1 名称: top_earlgrey
端口列表: kPortDeclarationList
找到 0 个端口定义

找到 624 个声明
  声明 1 类型: kPackageImportDeclaration
  声明 2 类型: kPackageImportDeclaration
  声明 3 类型: kPackageImportDeclaration

找到 1158 个语句/赋值
  语句 1 类型: kContinuousAssignmentStatement
  语句 2 类型: kContinuousAssignmentStatement
  语句 3 类型: kContinuousAssignmentStatement


In [4]:
# 查找模块实例化
def print_instance_info(node, indent=0):
    """打印模块实例化信息"""
    instance_nodes = node.find_all(lambda n: hasattr(n, 'tag') and n.tag == 'kGateInstance')
    
    if instance_nodes:
        print(f"{' ' * indent}找到 {len(instance_nodes)} 个模块实例化")
        # 限制输出数量
        for i, instance in enumerate(instance_nodes[:5]):
            # 查找实例化模块的类型名
            module_type = instance.find(lambda n: hasattr(n, 'tag') and n.tag == 'kGateInstanceType')
            type_name = "未找到"
            if module_type:
                type_id = module_type.find(lambda n: hasattr(n, 'tag') and n.tag == 'SymbolIdentifier')
                if type_id:
                    type_name = type_id.text
            
            # 查找实例名称
            instance_name = instance.find(lambda n: hasattr(n, 'tag') and n.tag == 'SymbolIdentifier')
            name = "未找到"
            if instance_name:
                name = instance_name.text
                
            print(f"{' ' * indent}  实例 {i+1}: 类型={type_name}, 名称={name}")
            
            # 查找连接的端口列表
            port_list = instance.find(lambda n: hasattr(n, 'tag') and (n.tag == 'kActualParameterList' or n.tag == 'kPortActualList'))
            if port_list:
                connections = port_list.find_all(lambda n: hasattr(n, 'tag') and n.tag == 'kActualNamedPort')
                print(f"{' ' * indent}    连接了 {len(connections)} 个命名端口")
                # 只显示前3个端口连接
                for j, conn in enumerate(connections[:3]):
                    port_name = conn.find(lambda n: hasattr(n, 'tag') and n.tag == 'SymbolIdentifier')
                    expr = conn.find(lambda n: hasattr(n, 'tag') and 'Expression' in n.tag)
                    if port_name:
                        print(f"{' ' * indent}      端口 {port_name.text} => {expr.text if expr else '?'}")
        
        # 如果实例太多，显示省略信息
        if len(instance_nodes) > 5:
            print(f"{' ' * indent}  ... 还有 {len(instance_nodes) - 5} 个实例 (已省略)")

# 查找模块中的参数定义
def print_params_info(node, indent=0):
    """打印模块参数信息"""
    param_nodes = node.find_all(lambda n: hasattr(n, 'tag') and 'Parameter' in n.tag)
    if param_nodes:
        print(f"{' ' * indent}找到 {len(param_nodes)} 个参数定义")
        for i, param in enumerate(param_nodes[:5]):
            param_name = param.find(lambda n: hasattr(n, 'tag') and n.tag == 'SymbolIdentifier')
            param_value = param.find(lambda n: hasattr(n, 'tag') and ('Expression' in n.tag or 'kConstant' in n.tag))
            if param_name:
                print(f"{' ' * indent}  参数 {i+1}: {param_name.text} = {param_value.text if param_value else '未找到值'}")
        
        if len(param_nodes) > 5:
            print(f"{' ' * indent}  ... 还有 {len(param_nodes) - 5} 个参数 (已省略)")

# 获取模块定义节点
module_nodes = anytree.findall(data.tree, lambda node: hasattr(node, 'tag') and node.tag == 'kModuleDeclaration')
if module_nodes:
    # 只检查第一个模块
    module = module_nodes[0]
    module_name = module.find(lambda n: hasattr(n, 'tag') and n.tag == 'SymbolIdentifier')
    print(f"模块名称: {module_name.text if module_name else '未找到'}")
    
    # 查找参数信息
    print("\n== 模块参数 ==")
    print_params_info(module)
    
    # 查找实例化信息
    print("\n== 模块实例化 ==")
    print_instance_info(module)
    
    # 查找always块
    always_blocks = module.find_all(lambda n: hasattr(n, 'tag') and 'kAlways' in n.tag)
    print(f"\n== Always 块 ==\n找到 {len(always_blocks)} 个 always 块")
    for i, block in enumerate(always_blocks[:3]):
        print(f"  Always块 {i+1} 类型: {block.tag}")
        
        # 查找敏感列表
        sens_list = block.find(lambda n: hasattr(n, 'tag') and n.tag == 'kSensitivityList')
        if sens_list:
            signals = sens_list.find_all(lambda n: hasattr(n, 'text') and n.text)
            print(f"    敏感信号列表: {', '.join([s.text for s in signals[:3]])}{', ...' if len(signals) > 3 else ''}")
        
        # 查找块中的语句
        statements = block.find_all(lambda n: hasattr(n, 'tag') and 'Statement' in n.tag)
        print(f"    包含 {len(statements)} 个语句")
    
    if len(always_blocks) > 3:
        print(f"  ... 还有 {len(always_blocks) - 3} 个 always 块 (已省略)")
    
    # 查找连续赋值语句
    assign_stmts = module.find_all(lambda n: hasattr(n, 'tag') and n.tag == 'kContinuousAssignmentStatement')
    print(f"\n== 连续赋值语句 ==\n找到 {len(assign_stmts)} 个连续赋值语句")
    for i, stmt in enumerate(assign_stmts[:3]):
        lhs = stmt.find(lambda n: hasattr(n, 'tag') and 'kNetVariableAssignment' in n.tag)
        if lhs:
            lhs_name = lhs.find(lambda n: hasattr(n, 'tag') and 'Identifier' in n.tag)
            lhs_text = lhs_name.text if lhs_name else "未找到"
            
            # 查找赋值表达式
            expr = lhs.find(lambda n: hasattr(n, 'tag') and 'Expression' in n.tag)
            expr_text = expr.text if expr else "未找到"
            
            print(f"  赋值 {i+1}: {lhs_text} = {expr_text}")
    
    if len(assign_stmts) > 3:
        print(f"  ... 还有 {len(assign_stmts) - 3} 个赋值 (已省略)")

模块名称: top_earlgrey

== 模块参数 ==
找到 86 个参数定义
  参数 1: SpiDeviceSramType = 1
  参数 2: SpiDeviceSramType = 1
  参数 3: AlertAsyncOn = alert_handler_reg_pkg::AsyncOn[0:0]
  参数 4: AlertAsyncOn = alert_handler_reg_pkg::AsyncOn[1:1]
  参数 5: AlertAsyncOn = alert_handler_reg_pkg::AsyncOn[2:2]
  ... 还有 81 个参数 (已省略)

== 模块实例化 ==
找到 45 个模块实例化
  实例 1: 类型=未找到, 名称=u_dft_tap_breakout
    连接了 8 个命名端口
      端口 req_i => pinmux_aon_dft_jtag_req
      端口 rsp_o => pinmux_aon_dft_jtag_rsp
      端口 tck_o => ?
  实例 2: 类型=未找到, 名称=u_uart0
    连接了 18 个命名端口
      端口 cio_rx_i => cio_uart0_rx_p2d
      端口 cio_tx_o => cio_uart0_tx_d2p
      端口 cio_tx_en_o => cio_uart0_tx_en_d2p
  实例 3: 类型=未找到, 名称=u_uart1
    连接了 18 个命名端口
      端口 cio_rx_i => cio_uart1_rx_p2d
      端口 cio_tx_o => cio_uart1_tx_d2p
      端口 cio_tx_en_o => cio_uart1_tx_en_d2p
  实例 4: 类型=未找到, 名称=u_uart2
    连接了 18 个命名端口
      端口 cio_rx_i => cio_uart2_rx_p2d
      端口 cio_tx_o => cio_uart2_tx_d2p
      端口 cio_tx_en_o => cio_uart2_tx_en_d2p
  实例 5: 类型=未找到, 名称=u_u

In [5]:
# 最关键的结构分析
import textwrap

# 查找模块层次结构，分析模块实例化关系
print("=== 模块层次结构分析 ===")

# 获取模块定义
module_nodes = anytree.findall(data.tree, lambda node: hasattr(node, 'tag') and node.tag == 'kModuleDeclaration')
if not module_nodes:
    print("未找到模块定义")
    
module = module_nodes[0]
module_name = module.find(lambda n: hasattr(n, 'tag') and n.tag == 'SymbolIdentifier')
print(f"顶层模块: {module_name.text if module_name else '未知'}")

# 寻找主要的模块实例化
instance_nodes = module.find_all(lambda n: hasattr(n, 'tag') and n.tag == 'kGateInstance')
print(f"顶层模块包含 {len(instance_nodes)} 个子模块实例")

print("\n主要子模块:")
# 按实例名称长短排序（通常主要模块名称更短）
instances_with_names = []
for instance in instance_nodes:
    inst_type = instance.find(lambda n: hasattr(n, 'tag') and n.tag == 'kGateInstanceType')
    if inst_type:
        type_id = inst_type.find(lambda n: hasattr(n, 'tag') and n.tag == 'SymbolIdentifier')
        type_name = type_id.text if type_id else "未知"
    else:
        type_name = "未知"
    
    inst_name = instance.find(lambda n: hasattr(n, 'tag') and n.tag == 'SymbolIdentifier')
    name = inst_name.text if inst_name else "未命名"
    instances_with_names.append((name, type_name, instance))

# 选取前10个重要的模块实例
important_instances = sorted(instances_with_names, key=lambda x: len(x[0]))[:10]
for name, type_name, instance in important_instances:
    print(f"  实例: {name} (类型: {type_name})")
    # 查找端口连接
    port_list = instance.find(lambda n: hasattr(n, 'tag') and (n.tag == 'kActualParameterList' or n.tag == 'kPortActualList'))
    if port_list:
        connections = port_list.find_all(lambda n: hasattr(n, 'tag') and n.tag == 'kActualNamedPort')
        critical_ports = [
            conn for conn in connections 
            if conn.find(lambda n: hasattr(n, 'tag') and n.tag == 'SymbolIdentifier') 
            and any(kw in conn.find(lambda n: hasattr(n, 'tag') and n.tag == 'SymbolIdentifier').text.lower() 
                   for kw in ['clk', 'rst', 'en', 'data', 'addr', 'intr', 'irq'])
        ]
        
        if critical_ports:
            print("    关键端口连接:")
            for conn in critical_ports[:3]:  # 只显示前3个关键端口
                port_name = conn.find(lambda n: hasattr(n, 'tag') and n.tag == 'SymbolIdentifier')
                expr = conn.find(lambda n: hasattr(n, 'tag') and ('Expression' in n.tag or 'Reference' in n.tag))
                if port_name and expr:
                    print(f"      {port_name.text} => {textwrap.shorten(expr.text, width=40)}")
    print()

# 寻找关键接口和连接
print("\n=== 关键信号和总线分析 ===")
# 查找所有的网线和变量声明
net_decls = module.find_all(lambda n: hasattr(n, 'tag') and ('kNetDeclaration' in n.tag or 'kDataDeclaration' in n.tag))
print(f"找到 {len(net_decls)} 个信号/变量声明")

# 提取可能是总线或关键信号的声明
bus_signals = []
for decl in net_decls:
    type_id = decl.find(lambda n: hasattr(n, 'tag') and (n.tag == 'kDataType' or n.tag == 'kNetType'))
    var_name = decl.find(lambda n: hasattr(n, 'tag') and n.tag == 'SymbolIdentifier')
    
    # 检查是否有数组维度（可能是总线）
    dimensions = decl.find_all(lambda n: hasattr(n, 'tag') and n.tag == 'kDimensionRange')
    
    if var_name and any(kw in var_name.text.lower() for kw in 
                    ['clk', 'rst', 'bus', 'data', 'addr', 'intr', 'irq', 'if_', 'tl_']):
        bus_type = ""
        if type_id:
            type_node = type_id.find(lambda n: hasattr(n, 'text') and n.text)
            bus_type = type_node.text if type_node else ""
        
        dimensions_str = ""
        if dimensions:
            dimensions_str = f"[{len(dimensions)}维数组]"
            
        bus_signals.append((var_name.text, bus_type, dimensions_str, decl))
        
# 显示可能的总线和关键信号
print("\n可能的总线或关键信号:")
for name, type_name, dim, decl in sorted(bus_signals, key=lambda x: x[0])[:15]:
    print(f"  {name} : {type_name} {dim}")

# 查找总线/接口的工作方式
print("\n=== 时钟域和复位信号分析 ===")
# 查找always块及其敏感信号列表
always_blocks = module.find_all(lambda n: hasattr(n, 'tag') and 'kAlways' in n.tag)

# 按敏感信号列表对always块进行分组
clock_domains = {}
for block in always_blocks:
    sens_list = block.find(lambda n: hasattr(n, 'tag') and n.tag == 'kSensitivityList')
    if sens_list:
        signals = sens_list.find_all(lambda n: hasattr(n, 'text') and n.text)
        signal_names = tuple(s.text for s in signals)
        if signal_names:
            if signal_names not in clock_domains:
                clock_domains[signal_names] = []
            clock_domains[signal_names].append(block)

# 输出各个时钟域信息
print(f"找到 {len(clock_domains)} 个潜在的时钟域")
for i, (signals, blocks) in enumerate(sorted(clock_domains.items(), key=lambda x: len(x[1]), reverse=True)[:5]):
    print(f"\n时钟域 {i+1}: 包含 {len(blocks)} 个always块")
    print(f"  敏感信号: {', '.join(signals)}")
    
    # 查看一个示例always块中的语句
    if blocks:
        sample_block = blocks[0]
        statements = sample_block.find_all(lambda n: hasattr(n, 'tag') and 'Statement' in n.tag)
        if_stmts = [s for s in statements if hasattr(s, 'tag') and s.tag == 'kIfStatement']
        
        if if_stmts:
            # 输出一个if语句的条件
            if_stmt = if_stmts[0]
            condition = if_stmt.find(lambda n: hasattr(n, 'tag') and 'Expression' in n.tag)
            if condition:
                print(f"  示例条件: {condition.text}")
                
                # 检查是否有复位逻辑
                if any(rst in condition.text.lower() for rst in ['rst', 'reset']):
                    print("  可能包含复位逻辑")

=== 模块层次结构分析 ===
顶层模块: top_earlgrey
顶层模块包含 45 个子模块实例

主要子模块:
  实例: u_aes (类型: 未知)
    关键端口连接:
      lc_escalate_en_i => lc_ctrl_lc_escalate_en
      clk_i => clkmgr_aon_clocks.clk_main_aes
      clk_edn_i => clkmgr_aon_clocks.clk_main_aes

  实例: u_gpio (类型: 未知)
    关键端口连接:
      cio_gpio_en_o => cio_gpio_gpio_en_d2p
      intr_gpio_o => intr_gpio_gpio
      clk_i => clkmgr_aon_clocks.clk_io_div4_peri

  实例: u_i2c0 (类型: 未知)
    关键端口连接:
      cio_sda_en_o => cio_i2c0_sda_en_d2p
      cio_scl_en_o => cio_i2c0_scl_en_d2p
      intr_fmt_threshold_o => intr_i2c0_fmt_threshold

  实例: u_i2c1 (类型: 未知)
    关键端口连接:
      cio_sda_en_o => cio_i2c1_sda_en_d2p
      cio_scl_en_o => cio_i2c1_scl_en_d2p
      intr_fmt_threshold_o => intr_i2c1_fmt_threshold

  实例: u_i2c2 (类型: 未知)
    关键端口连接:
      cio_sda_en_o => cio_i2c2_sda_en_d2p
      cio_scl_en_o => cio_i2c2_scl_en_d2p
      intr_fmt_threshold_o => intr_i2c2_fmt_threshold

  实例: u_hmac (类型: 未知)
    关键端口连接:
      intr_hmac_done_o => intr_hmac_hmac_d

In [6]:
# 工具函数，用于SystemVerilog语法树分析
def sv_syntax_tree_analyzer(tree, focus_area=None):
    """
    SystemVerilog语法树分析器 - 提供语法树结构的综合分析
    
    Args:
        tree: 要分析的语法树节点
        focus_area: 可选的重点分析区域，如 'module', 'interface', 'always', 'assign'
        
    Returns:
        dict: 包含分析结果的字典
    """
    result = {
        "structure_summary": {},
        "node_types": {},
        "interesting_nodes": {}
    }
    
    # 1. 基本结构摘要
    if hasattr(tree, 'tag'):
        result["structure_summary"]["root_tag"] = tree.tag
        
    # 分析节点类型统计
    node_types = {}
    for node in anytree.iterators.PreOrderIter(tree):
        if hasattr(node, 'tag'):
            if node.tag not in node_types:
                node_types[node.tag] = 0
            node_types[node.tag] += 1
    
    # 按出现频率排序
    result["node_types"] = dict(sorted(node_types.items(), key=lambda x: x[1], reverse=True))
    
    # 2. 根据关注点进行特定分析
    if focus_area == 'module' or focus_area is None:
        # 模块定义和实例化
        module_nodes = anytree.findall(tree, lambda n: hasattr(n, 'tag') and n.tag == 'kModuleDeclaration')
        module_instances = anytree.findall(tree, lambda n: hasattr(n, 'tag') and n.tag == 'kGateInstance')
        
        result["interesting_nodes"]["modules"] = {
            "declarations": len(module_nodes),
            "instances": len(module_instances)
        }
    
    if focus_area == 'always' or focus_area is None:
        # Always块
        always_blocks = anytree.findall(tree, lambda n: hasattr(n, 'tag') and 'kAlways' in n.tag)
        
        # 统计不同类型的always块
        always_types = {}
        for block in always_blocks:
            if block.tag not in always_types:
                always_types[block.tag] = 0
            always_types[block.tag] += 1
            
        result["interesting_nodes"]["always_blocks"] = {
            "total": len(always_blocks),
            "types": always_types
        }
    
    if focus_area == 'assign' or focus_area is None:
        # 赋值语句
        assign_stmts = anytree.findall(tree, lambda n: hasattr(n, 'tag') and 
                                     (n.tag == 'kContinuousAssignmentStatement' or 'Assignment' in n.tag))
        result["interesting_nodes"]["assignments"] = len(assign_stmts)
    
    if focus_area == 'declarations' or focus_area is None:
        # 各类声明
        var_decls = anytree.findall(tree, lambda n: hasattr(n, 'tag') and 'Declaration' in n.tag)
        result["interesting_nodes"]["declarations"] = len(var_decls)
    
    return result

# 对当前语法树运行分析器
print("=== SystemVerilog 语法树综合分析 ===\n")
analysis = sv_syntax_tree_analyzer(data.tree)

print("1. 节点类型统计 (Top 10):")
for i, (tag, count) in enumerate(list(analysis["node_types"].items())[:10]):
    print(f"  {tag}: {count}")

print("\n2. 关键结构统计:")
if "modules" in analysis["interesting_nodes"]:
    modules = analysis["interesting_nodes"]["modules"]
    print(f"  模块定义: {modules['declarations']}")
    print(f"  模块实例: {modules['instances']}")

if "always_blocks" in analysis["interesting_nodes"]:
    always = analysis["interesting_nodes"]["always_blocks"]
    print(f"  Always块: {always['total']}")
    if "types" in always:
        for atype, count in always["types"].items():
            print(f"    - {atype}: {count}")

if "assignments" in analysis["interesting_nodes"]:
    print(f"  赋值语句: {analysis['interesting_nodes']['assignments']}")

if "declarations" in analysis["interesting_nodes"]:
    print(f"  各类声明: {analysis['interesting_nodes']['declarations']}")

print("\n=== 语法树结构分析总结 ===")
print("""
基于上述的语法树分析，在研究SystemVerilog代码的语法树时，您应重点关注：

1. 模块定义和层次结构:
   - kModuleDeclaration: 定义一个模块
   - kGateInstance: 实例化子模块
   - kPortDeclarationList/kPort: 模块端口定义

2. 信号和变量声明:
   - kNetDeclaration: 网线类型信号声明(如wire)
   - kDataDeclaration: 数据类型声明(如reg, logic)
   - kDimensionRange: 数组维度，用于识别总线宽度

3. 行为建模:
   - kAlways*/kAlwaysStatement: always块，表示时序逻辑或组合逻辑
   - kSensitivityList: 敏感信号列表，表示时钟域
   - kIfStatement/kCaseStatement: 条件语句，常用于状态机和条件逻辑

4. 数据流建模:
   - kContinuousAssignmentStatement: 连续赋值语句，组合逻辑
   - kNetVariableAssignment: 网线赋值

5. 层次访问:
   - 使用anytree库的find/findall方法可以高效定位特定类型的节点
   - 结合Node.tag属性和node.text属性可以识别特定节点及其内容

对于不同目的的分析，可以选择性地关注不同的节点类型。
""")

=== SystemVerilog 语法树综合分析 ===

1. 节点类型统计 (Top 10):
  SymbolIdentifier: 5751
  kUnqualifiedId: 3845
  kExpression: 2936
  kLocalRoot: 2917
  kReference: 2579
  kFunctionCall: 2193
  .: 1568
  ,: 1489
  kParenGroup: 1341
  (: 1341

2. 关键结构统计:
  模块定义: 1
  模块实例: 45
  Always块: 0
  赋值语句: 1158
  各类声明: 786

=== 语法树结构分析总结 ===

基于上述的语法树分析，在研究SystemVerilog代码的语法树时，您应重点关注：

1. 模块定义和层次结构:
   - kModuleDeclaration: 定义一个模块
   - kGateInstance: 实例化子模块
   - kPortDeclarationList/kPort: 模块端口定义

2. 信号和变量声明:
   - kNetDeclaration: 网线类型信号声明(如wire)
   - kDataDeclaration: 数据类型声明(如reg, logic)
   - kDimensionRange: 数组维度，用于识别总线宽度

3. 行为建模:
   - kAlways*/kAlwaysStatement: always块，表示时序逻辑或组合逻辑
   - kSensitivityList: 敏感信号列表，表示时钟域
   - kIfStatement/kCaseStatement: 条件语句，常用于状态机和条件逻辑

4. 数据流建模:
   - kContinuousAssignmentStatement: 连续赋值语句，组合逻辑
   - kNetVariableAssignment: 网线赋值

5. 层次访问:
   - 使用anytree库的find/findall方法可以高效定位特定类型的节点
   - 结合Node.tag属性和node.text属性可以识别特定节点及其内容

对于不同目的的分析，可以选择性地关注不同的节点类型。



# SystemVerilog 语法树节点类型详解

通过对Verible语法树的分析，我们可以看到语法树由多种类型的节点组成，每种类型对应SystemVerilog语言的不同语法结构。以下是对主要节点类型的详细说明：

## 节点类型分类

SystemVerilog语法树中的节点类型可以大致分为以下几类：

### 1. 模块相关节点 

这类节点定义了设计的基本结构和层次关系：

* **`kModuleDeclaration`** - 模块定义，是任何SystemVerilog设计的基础单元
* **`kGateInstance`** - 模块实例化，用于在一个模块中引用另一个模块
* **`kPortDeclarationList`** - 模块端口列表，定义模块的输入和输出
* **`kPort`** - 单个端口定义
* **`kModuleItemList`** - 模块内部项列表，包含模块内的所有声明和语句
* **`kActualParameterList`/`kPortActualList`** - 实例化时的参数和端口连接列表

### 2. 声明相关节点 

这类节点用于声明变量、类型和参数：

* **`kNetDeclaration`** - 网线类型信号声明（例如 wire, tri）
* **`kDataDeclaration`** - 数据类型声明（例如 logic, reg, integer）
* **`kParameterDeclaration`** - 参数声明
* **`kLocalparamDeclaration`** - 本地参数声明
* **`kDimensionRange`** - 数组维度，用于定义总线宽度

### 3. 行为建模节点

这类节点描述时序逻辑和组合逻辑的行为：

* **`kAlwaysStatement`/`kAlwaysCombStatement`/`kAlwaysFFStatement`** - always块，定义时序或组合逻辑
* **`kSensitivityList`** - 敏感信号列表，定义触发always块的条件
* **`kIfStatement`** - if条件语句，用于条件分支
* **`kCaseStatement`/`kCaseInsideStatement`** - case选择语句，常用于状态机实现
* **`kForStatement`/`kWhileStatement`** - 循环语句，用于重复操作

### 4. 数据流建模节点

这类节点用于描述数据的流动和变换：

* **`kContinuousAssignmentStatement`** - 连续赋值语句，用于组合逻辑
* **`kNetVariableAssignment`** - 网线变量赋值
* **`kBlockingAssignmentStatement`** - 阻塞赋值（=）
* **`kNonblockingAssignmentStatement`** - 非阻塞赋值（<=）

### 5. 表达式相关节点

这类节点表示各种表达式和运算：

* **`kExpression`** - 一般表达式
* **`kBinaryExpression`** - 二元表达式（例如 a+b, a&b）
* **`kUnaryPrefixExpression`** - 前缀一元表达式（例如 !a, ~b）
* **`kReferenceExpression`** - 引用表达式

### 6. 标识符和常量节点

这类节点表示名称和字面值：

* **`SymbolIdentifier`** - 符号标识符，用于变量名、模块名等
* **`kConstant`** - 常量值
* **`NumericLiteral`** - 数字字面值

## 重点关注的节点类型

在进行SystemVerilog代码分析时，以下几类节点最值得重点关注：

1. **模块层次结构节点**：
   * `kModuleDeclaration` - 理解设计的基本单元
   * `kGateInstance` - 分析模块之间的层次和连接关系
   * `kPort`/`kPortDeclarationList` - 了解模块接口

2. **信号和变量声明节点**：
   * `kNetDeclaration`/`kDataDeclaration` - 分析信号和变量的类型和范围
   * `kDimensionRange` - 理解总线宽度和数组维度

3. **时序逻辑节点**：
   * `kAlwaysFFStatement` - 识别时序逻辑块
   * `kSensitivityList` - 确定时钟域
   * `kNonblockingAssignmentStatement` - 分析时序逻辑中的数据流动

4. **组合逻辑节点**：
   * `kAlwaysCombStatement`/`kContinuousAssignmentStatement` - 识别组合逻辑
   * `kIfStatement`/`kCaseStatement` - 理解条件逻辑和数据选择

5. **状态机相关节点**：
   * `kEnumDeclaration` - 识别状态定义
   * `kCaseStatement` + `kAlwaysFFStatement` - 找出状态机的状态转移逻辑

## 使用这些节点类型进行设计分析

通过分析这些节点的结构和关系，可以：

1. **构建模块层次关系图** - 通过分析`kModuleDeclaration`和`kGateInstance`节点
2. **识别关键信号流** - 通过追踪`SymbolIdentifier`在赋值语句中的使用
3. **划分时钟域** - 通过分析`kAlwaysFFStatement`中的`kSensitivityList`
4. **提取状态机** - 通过识别`kEnumDeclaration`和特定模式的`kCaseStatement`
5. **识别操作时序** - 通过分析`kNonblockingAssignmentStatement`的条件和依赖关系

正确理解这些节点类型及其结构，是高效分析和处理SystemVerilog代码的关键。

In [7]:
# 统计并分析语法树中的节点类型
def analyze_node_types(tree):
    """分析语法树中的节点类型及其出现次数"""
    node_types = {}
    for node in anytree.iterators.PreOrderIter(tree):
        if hasattr(node, 'tag'):
            if node.tag not in node_types:
                node_types[node.tag] = 0
            node_types[node.tag] += 1
    
    return node_types

# 获取所有节点类型及其计数
node_types = analyze_node_types(data.tree)

# 按出现频率排序
sorted_types = sorted(node_types.items(), key=lambda x: x[1], reverse=True)

# 显示前30种最常见的节点类型
print(f"在当前语法树中找到 {len(node_types)} 种不同类型的节点")
print("\n最常见的30种节点类型:")
for i, (tag, count) in enumerate(sorted_types[:30]):
    print(f"{i+1:2d}. {tag:40s}: {count:5d}")

# 根据名称对节点类型进行分类
def categorize_node_types(node_types):
    categories = {
        "模块相关": [],
        "声明相关": [],
        "赋值相关": [],
        "行为相关": [],
        "表达式相关": [],
        "标识符相关": [],
        "其他": []
    }
    
    for node_type in node_types:
        if any(x in node_type.lower() for x in ['module', 'port', 'gate', 'instance']):
            categories["模块相关"].append(node_type)
        elif any(x in node_type.lower() for x in ['decl', 'declaration', 'parameter', 'type']):
            categories["声明相关"].append(node_type)
        elif any(x in node_type.lower() for x in ['assign', 'assignment']):
            categories["赋值相关"].append(node_type)
        elif any(x in node_type.lower() for x in ['always', 'statement', 'if', 'case', 'for', 'while']):
            categories["行为相关"].append(node_type)
        elif any(x in node_type.lower() for x in ['expr', 'expression', 'operator', 'constant']):
            categories["表达式相关"].append(node_type)
        elif any(x in node_type.lower() for x in ['identifier', 'symbol', 'name']):
            categories["标识符相关"].append(node_type)
        else:
            categories["其他"].append(node_type)
    
    return categories

# 对节点类型进行分类
categories = categorize_node_types(node_types.keys())

print("\n=== 节点类型分类 ===")
for category, types in categories.items():
    if types:
        print(f"\n{category} ({len(types)}):")
        for t in sorted(types)[:10]:  # 每类只显示前10个
            print(f"  - {t} ({node_types[t]})")
        if len(types) > 10:
            print(f"  ... 还有 {len(types) - 10} 种类型 (已省略)")

在当前语法树中找到 94 种不同类型的节点

最常见的30种节点类型:
 1. SymbolIdentifier                        :  5751
 2. kUnqualifiedId                          :  3845
 3. kExpression                             :  2936
 4. kLocalRoot                              :  2917
 5. kReference                              :  2579
 6. kFunctionCall                           :  2193
 7. .                                       :  1568
 8. ,                                       :  1489
 9. kParenGroup                             :  1341
10. (                                       :  1341
11. )                                       :  1340
12. kActualNamedPort                        :  1109
13. ;                                       :   980
14. [                                       :   885
15. ]                                       :   885
16. kNumber                                 :   747
17. TK_DecNumber                            :   736
18. kUnpackedDimensions                     :   727
19. kDimensionScalar        

In [8]:
# 深入分析关键节点的结构和用途
def examine_node_structure(tree, node_tag, limit=3):
    """查找特定类型的节点并分析其结构"""
    nodes = anytree.findall(tree, lambda n: hasattr(n, 'tag') and n.tag == node_tag)
    if not nodes:
        print(f"未找到 {node_tag} 类型的节点")
        return
    
    print(f"找到 {len(nodes)} 个 {node_tag} 类型的节点")
    
    # 只分析前limit个节点
    for i, node in enumerate(nodes[:limit]):
        print(f"\n节点 {i+1}:")
        
        # 打印节点的直接子节点
        children = node.children if hasattr(node, 'children') else []
        if children:
            print(f"  子节点类型 ({len(children)}):")
            for j, child in enumerate(children):
                if hasattr(child, 'tag'):
                    print(f"    {j+1}. {child.tag}")
                else:
                    print(f"    {j+1}. 无tag属性")
        else:
            print("  无子节点")
        
        # 尝试获取节点的文本内容
        if hasattr(node, 'text') and node.text:
            print(f"  文本内容: '{node.text}'")

# 分析几个重要的节点类型
important_node_types = [
    'kModuleDeclaration',       # 模块定义
    'kGateInstance',            # 模块实例化
    'kAlwaysStatement',         # always块
    'kContinuousAssignmentStatement',  # 连续赋值
    'kNetDeclaration',          # 网线声明
    'kDataDeclaration',         # 数据类型声明
    'kIfStatement',             # if语句
    'kCaseStatement'            # case语句
]

print("\n=== 重要节点类型结构分析 ===")
for node_type in important_node_types:
    print(f"\n## {node_type} 结构分析:")
    examine_node_structure(data.tree, node_type, 2)


=== 重要节点类型结构分析 ===

## kModuleDeclaration 结构分析:
找到 1 个 kModuleDeclaration 类型的节点

节点 1:
  子节点类型 (4):
    1. kModuleHeader
    2. kModuleItemList
    3. endmodule
    4. 无tag属性
  文本内容: 'module top_earlgrey #(
  // Manually defined parameters

  // Auto-inferred parameters
  // parameters for uart0
  // parameters for uart1
  // parameters for uart2
  // parameters for uart3
  // parameters for gpio
  parameter bit GpioGpioAsyncOn = 1,
  // parameters for spi_device
  parameter spi_device_pkg::sram_type_e SpiDeviceSramType = spi_device_pkg::DefaultSramType,
  // parameters for i2c0
  parameter int I2c0InputDelayCycles = 0,
  // parameters for i2c1
  parameter int I2c1InputDelayCycles = 0,
  // parameters for i2c2
  parameter int I2c2InputDelayCycles = 0,
  // parameters for pattgen
  // parameters for rv_timer
  // parameters for otp_ctrl
  parameter OtpCtrlMemInitFile = "",
  // parameters for lc_ctrl
  parameter bit SecLcCtrlVolatileRawUnlockEn = top_pkg::SecVolatileRawUnlockEn,
  para

In [9]:
# 实用查询工具：查找节点并显示其用法示例

def find_node_example(tree, search_pattern, limit=3):
    """
    查找包含特定模式的节点类型，并显示示例
    
    Args:
        tree: 语法树根节点
        search_pattern: 字符串模式，用于匹配节点类型名称
        limit: 最多返回的示例数量
    """
    # 查找匹配的节点类型
    matching_types = []
    for node in anytree.iterators.PreOrderIter(tree):
        if hasattr(node, 'tag') and search_pattern.lower() in node.tag.lower():
            if node.tag not in matching_types:
                matching_types.append(node.tag)
    
    if not matching_types:
        print(f"没有找到包含 '{search_pattern}' 的节点类型")
        return
    
    print(f"找到 {len(matching_types)} 种包含 '{search_pattern}' 的节点类型:")
    
    # 显示每种类型的示例
    for node_type in matching_types:
        print(f"\n## {node_type}:")
        
        # 查找这种类型的节点
        nodes = anytree.findall(tree, lambda n: hasattr(n, 'tag') and n.tag == node_type)
        print(f"  总共有 {len(nodes)} 个实例")
        
        # 显示前limit个示例
        for i, node in enumerate(nodes[:limit]):
            print(f"\n  示例 {i+1}:")
            
            # 尝试获取文本内容
            if hasattr(node, 'text') and node.text:
                # 如果文本太长，只显示前100个字符
                text = node.text
                if len(text) > 100:
                    text = text[:97] + "..."
                print(f"    文本: {text}")
            
            # 获取父节点上下文
            parent = node.parent
            if parent and hasattr(parent, 'tag'):
                print(f"    父节点: {parent.tag}")
                
            # 显示子节点
            children = node.children if hasattr(node, 'children') else []
            if children:
                print(f"    子节点 ({len(children)}):")
                for j, child in enumerate(children[:3]):
                    if hasattr(child, 'tag'):
                        child_text = child.text if hasattr(child, 'text') and child.text else ""
                        if len(child_text) > 30:
                            child_text = child_text[:27] + "..."
                        print(f"      {j+1}. {child.tag} {f'({child_text})' if child_text else ''}")
                if len(children) > 3:
                    print(f"      ... 还有 {len(children) - 3} 个子节点")

print("您可以通过修改下面的search_pattern来查询特定类型的节点:")
find_node_example(data.tree, "assign", 2)  # 修改"assign"为您要查询的节点类型模式

您可以通过修改下面的search_pattern来查询特定类型的节点:
找到 5 种包含 'assign' 的节点类型:

## kTrailingAssign:
  总共有 67 个实例

  示例 1:
    文本: = 1
    父节点: kParamDeclaration
    子节点 (3):
      1. = (=)
      2. kExpression (1)

  示例 2:
    文本: = spi_device_pkg::DefaultSramType
    父节点: kParamDeclaration
    子节点 (3):
      1. = (=)
      2. kExpression (spi_device_pkg::DefaultSram...)

## kContinuousAssignmentStatement:
  总共有 386 个实例

  示例 1:
    文本: assign edn0_edn_req[2] = ast_edn_req_i;
    父节点: kModuleItemList
    子节点 (5):
      1. assign (assign)
      ... 还有 2 个子节点

  示例 2:
    文本: assign ast_edn_rsp_o = edn0_edn_rsp[2];
    父节点: kModuleItemList
    子节点 (5):
      1. assign (assign)
      ... 还有 2 个子节点

## assign:
  总共有 386 个实例

  示例 1:
    文本: assign
    父节点: kContinuousAssignmentStatement

  示例 2:
    文本: assign
    父节点: kContinuousAssignmentStatement

## kAssignmentList:
  总共有 386 个实例

  示例 1:
    文本: edn0_edn_req[2] = ast_edn_req_i
    父节点: kContinuousAssignmentStatement
    子节点 (1):
      1. kNetVariableAs

# SystemVerilog语法树节点类型总结

## 最重要的节点类型及其用途

在分析SystemVerilog设计时，不同类型的分析任务需要关注不同的节点类型：

### 设计层次和结构分析
- **`kModuleDeclaration`**: 模块定义，设计的基本单元
- **`kGateInstance`**: 模块实例化，形成设计的层次结构
- **`kPortDeclarationList`/`kPort`**: 模块接口定义
- **`kActualNamedPort`**: 实例化时的端口连接，用于分析信号流

### 数据流分析
- **`kContinuousAssignmentStatement`**: 连续赋值，表示组合逻辑
- **`kBlockingAssignmentStatement`/`kNonblockingAssignmentStatement`**: 代码块内的赋值，区分时序和非时序逻辑
- **`kNetVariableAssignment`**: 网线变量赋值，是赋值语句的一部分

### 时序分析
- **`kAlwaysFFStatement`**: 描述时序逻辑的always块
- **`kSensitivityList`**: 敏感信号列表，定义时钟域
- **`kEdgeDescriptor`**: 边沿描述符（如posedge, negedge）

### 状态机和控制逻辑
- **`kIfStatement`/`kElseClause`**: 条件分支
- **`kCaseStatement`/`kCaseItem`**: 多路选择，常用于状态机
- **`kEnumDeclaration`**: 枚举定义，常用于定义状态

### 数据类型和存储
- **`kNetDeclaration`/`kDataDeclaration`**: 信号和变量声明
- **`kDimensionRange`**: 定义数组维度和总线宽度
- **`kDataType`/`kNetType`**: 数据类型信息

## 节点类型的组合模式

某些特定的功能在语法树中通常表现为特定节点类型的组合：

1. **时序逻辑模式**: 
   ```
   kAlwaysFFStatement
     ├─ kSensitivityList (clk, rst)
     └─ kIfStatement (复位逻辑)
         ├─ kBlockingAssignmentStatement (复位值)
         └─ kElseClause
             └─ kNonblockingAssignmentStatement (时序值)
   ```

2. **组合逻辑模式**:
   ```
   kAlwaysCombStatement 或 kContinuousAssignmentStatement
     └─ 各种赋值和表达式
   ```

3. **有限状态机模式**:
   ```
   kEnumDeclaration (状态定义)
   kAlwaysFFStatement (状态寄存器)
   kAlwaysCombStatement
     └─ kCaseStatement (状态转移逻辑)
   ```

## 如何高效使用节点类型

1. **使用适当的过滤条件**：基于节点的tag属性，使用anytree的find和findall方法定位特定类型的节点
2. **组合多种节点类型**：例如，结合`kAlwaysFFStatement`和`kSensitivityList`识别时钟域
3. **浏览节点的层次结构**：使用node.parent和node.children理解节点间的关系
4. **利用文本内容**：node.text属性提供了节点对应的源代码文本

熟悉这些关键节点类型及其用途，将极大地提高您分析和处理SystemVerilog代码的能力。